<a href="https://colab.research.google.com/github/simon-clematide/colab-notebooks-for-teaching/blob/main/notebooks/topic_modeling_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup




In [49]:
#! pip install --upgrade pandas-profiling jupyter
! pip install pyldavis==3.4.0

In [50]:
# try to avoid warnings but not really working for now
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="ipykernel.ipkernel")


# `pyLDAvis`

pyLDAvis now also supports LDA application from scikit-learn. Let's take a look into this in more detail. We will be using the 20 newsgroups dataset as provided by scikit-learn.

In [51]:
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

## Load 20 newsgroups dataset

First, the 20 newsgroups dataset available in sklearn is loaded. As always, the headers, footers and quotes are removed.

Newsgroup categories:
`['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']`

In [52]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [53]:
cats = ['sci.med', 'alt.atheism', 'rec.autos', 'sci.space','rec.sport.baseball']
newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'),categories=cats)
docs_raw = newsgroups.data
print(len(docs_raw))

2858


In [54]:
print(docs_raw[72])

No, he's not nuts, WIP is second to none THE sports station.  They
don't have Tony Bruno working ESPN radio and Al Morganti doing Friday
Night Hockey because they suck.  I live in Richmond Va, but I visit
Phila often, and on the way I get WTEM Washington) and WIP.  I hear
the FAN at night wherever I go (the signal used to be WNBC, when they
played golden oldies) because you can't avoid it.  Of those three,
WIP has the best hosts hands down.  Chuck Cooperstein isn't a homer,
and neither is Jody Mac.  WTEM is too generic to be placed in the
catergory.  In fact if you have heard WTEM and the FAN you notice the
theme music is identical...same ownership?? I think so!  WIP is
totally original.  Their hosts actually have a personality (this is a
knock at TEM (the TEAM) not the FAN because Mike and the Mad Dog and
Sommers are good) I mean comparing the morning guys in Philadelphia
to the ones in Washington is a total joke.  Anyway, I like the FAN
and WIP, but I think the edge goes to 'IP.  

W

## Convert to document-term matrix

Next, the raw documents are converted into document-term matrix, possibly as raw counts or in TF-IDF form.

In [55]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5,  # exclude words with a relative document frequency greater than 50%
                                min_df = 10    # exclude tokens that occur less than 10 times
                                )
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(2858, 3234)


In [56]:
# How does a certain document look like in this representation?
# Get the mapping of column indices to vocabulary items
index2vocabulary_item = tf_vectorizer.get_feature_names_out()

# Get the dense matrix representation of the document-term matrix
doc_index = 72  # Index of the document to show
doc_matrix = dtm_tf.getrow(doc_index).toarray()

# Print the words and their counts in the document
for i, count in enumerate(doc_matrix[0]):
    if count > 0:
        word = index2vocabulary_item[i]
        print(f"{word}: {count}")

actually: 1
admit: 1
avoid: 1
away: 1
basically: 1
believe: 1
best: 2
bet: 1
blown: 1
book: 1
cause: 1
chuck: 1
cobb: 2
comparing: 1
control: 1
days: 1
die: 1
doing: 2
don: 1
edge: 1
fact: 1
fan: 6
fans: 1
finally: 1
friday: 1
games: 2
glad: 1
goes: 1
golden: 1
good: 1
got: 1
guy: 1
guys: 2
hands: 1
hard: 1
hear: 2
heard: 3
homer: 1
identical: 1
inches: 1
isn: 1
joke: 1
knock: 2
knows: 1
like: 3
line: 1
listening: 1
live: 1
mac: 1
mad: 2
mean: 1
mike: 1
morning: 1
music: 1
national: 1
night: 2
notice: 1
nuts: 1
ones: 1
original: 1
philadelphia: 1
phillies: 3
phone: 1
placed: 1
play: 2
played: 1
radio: 1
really: 1
remember: 1
richmond: 2
right: 1
said: 1
san: 1
second: 1
signal: 1
sports: 3
started: 1
station: 4
steve: 1
strong: 1
suck: 1
summer: 1
team: 1
think: 2
thought: 1
tony: 1
total: 1
totally: 2
used: 1
usually: 1
visit: 1
washington: 2
wasn: 1
way: 1
weekend: 1
went: 3
win: 1
working: 1
year: 1


Alternative, we can build a tf-idf document-term matrix

In [57]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

(2858, 3234)


In [58]:
# How does a certain document look like in this representation?
# Get the mapping of column indices to vocabulary items
index2vocabulary_item = tfidf_vectorizer.get_feature_names_out()

# Get the dense matrix representation of the document-term matrix
doc_index = 72  # Index of the document to show
doc_matrix = dtm_tfidf.getrow(doc_index).toarray()

# Print the words and their counts in the document
for i, count in enumerate(doc_matrix[0]):
    if count > 0:
        word = index2vocabulary_item[i]
        print(f"{word}: {count}")

actually: 0.04904840759498951
admit: 0.07299497510211472
avoid: 0.06545807155396616
away: 0.05456901558145122
basically: 0.06478407157709502
believe: 0.04675876453764755
best: 0.0994820820268445
bet: 0.07670547711801053
blown: 0.08267003006955992
book: 0.05844788921599561
cause: 0.054878639478271374
chuck: 0.08651485315372326
cobb: 0.16890930987684408
comparing: 0.08037327994808072
control: 0.05929226473552773
days: 0.05519560691471015
die: 0.07426877981418571
doing: 0.10794098035118706
don: 0.03345256161085763
edge: 0.08185979207508924
fact: 0.05069871139558411
fan: 0.4015050116497602
fans: 0.07009394406678396
finally: 0.06641310873076137
friday: 0.07903568115882252
games: 0.11800967575437019
glad: 0.07670547711801053
goes: 0.0601936062442923
golden: 0.0854450956691524
good: 0.03852922757106624
got: 0.048135332254219995
guy: 0.062202102910416196
guys: 0.1383003040359028
hands: 0.074721868417729
hard: 0.056079554382427656
hear: 0.12588956947024332
heard: 0.16722355825998003
homer: 0.08

## Fit Latent Dirichlet Allocation models

Finally, the LDA models are fitted. n_components is number of topics.

In [59]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0,verbose=1, max_iter=10)
lda_tf.fit(dtm_tf)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(random_state=0, verbose=1)

## Visualizing the models with pyLDAvis
Multidimensional scaling = Dimension reduction

Can you reidentify the newsgroups? `['sci.med', 'alt.atheism', 'rec.autos', 'sci.space','rec.sport.baseball']`

Hover over topics circles and terms to explore the connection between words and topics...

In [65]:
pyLDAvis.lda_model.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.211854  0.059024       1        1  18.910916
3     -0.090269 -0.087087       2        1  17.486832
8     -0.083085  0.082379       3        1  14.880483
4     -0.178631  0.050412       4        1  11.391817
7      0.172366  0.025299       5        1   8.397854
9     -0.013638 -0.001099       6        1   7.465159
0     -0.102297 -0.010045       7        1   7.015526
2     -0.068277  0.115510       8        1   5.509059
6      0.123235  0.049783       9        1   5.156471
5      0.028742 -0.284177      10        1   3.785882, topic_info=       Term         Freq        Total Category  logprob  loglift
896     edu   614.000000   614.000000  Default  30.0000  30.0000
2724  space  1001.000000  1001.000000  Default  29.0000  29.0000
523     com   359.000000   359.000000  Default  28.0000  28.0000
399     car   524.000000   524.000000  Default  27.0000  27.0000
1216    god   425.000000   425.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
1714  makes    25.032935   152.088429  Topic10  -5.4689   1.4696
835    does    28.929816   568.808166  Topic10  -5.3242   0.2952
1230    gov    22.784822   108.915916  Topic10  -5.5630   1.7094
1516   john    23.085518   122.473569  Topic10  -5.5499   1.6052
3039    try    21.894465   189.121446  Topic10  -5.6028   1.1177

[665 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
3         1  0.025447           abs
3         3  0.941547           abs
9         1  0.176406  acceleration
9         3  0.141125  acceleration
9         8  0.035281  acceleration
...     ...       ...           ...
3226      6  0.985435         yeast
3229      1  0.077083          york
3229      4  0.067447          york
3229      5  0.144530          york
3229      8  0.713014          york

[2017 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 9, 5, 8, 10, 1, 3, 7, 6])

## Topix Modeling with TFIDF values

In [61]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0, verbose=1,max_iter=10)
lda_tfidf.fit(dtm_tfidf)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(random_state=0, verbose=1)

In [62]:
pyLDAvis.lda_model.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.036409 -0.150971       1        1  36.003562
4      0.147098  0.043129       2        1  20.748950
2      0.130394  0.029769       3        1  15.690256
1     -0.064692  0.157209       4        1  10.466252
7     -0.095185 -0.050091       5        1   3.851599
5     -0.045482  0.001358       6        1   3.017606
8     -0.031716  0.002176       7        1   2.788787
3     -0.019594 -0.014178       8        1   2.613300
0     -0.029215 -0.008849       9        1   2.531343
6     -0.028016 -0.009553      10        1   2.288346, topic_info=          Term       Freq      Total Category  logprob  loglift
399        car  36.000000  36.000000  Default  30.0000  30.0000
896        edu  27.000000  27.000000  Default  29.0000  29.0000
1226    gordon  11.000000  11.000000  Default  28.0000  28.0000
246      banks  11.000000  11.000000  Default  27.0000  27.0000
2711      soon  12.000000  12.000000  Default  26.0000  26.0000
...        ...        ...        ...      ...      ...      ...
3226     yeast   0.739740   3.742208  Topic10  -6.0145   2.1562
2557       say   1.198053  25.033428  Topic10  -5.5323   0.7378
32    actually   0.946476  18.228822  Topic10  -5.7680   0.8193
483       city   0.771083   6.891849  Topic10  -5.9730   1.5870
3199      work   0.817792  17.792682  Topic10  -5.9141   0.6974

[502 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
9         4  0.816638  acceleration
16        7  0.560407     accidents
23        1  0.867170           act
32        1  0.493724      actually
32        2  0.274291      actually
...     ...       ...           ...
3225      3  0.215005         years
3226      3  0.267222         yeast
3226      6  0.534444         yeast
3226     10  0.267222         yeast
3232      8  0.622199         yount

[847 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 5, 3, 2, 8, 6, 9, 4, 1, 7])

### Using different MDS functions

With `sklearn` installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default PCoA is not satisfactory.

In [63]:
pyLDAvis.lda_model.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.050207  0.300478       1        1  18.910916
3      0.150495 -0.161884       2        1  17.486832
8      0.081503  0.135539       3        1  14.880483
4     -0.165656 -0.191262       4        1  11.391817
7     -0.224373  0.180753       5        1   8.397854
9      0.007855  0.007114       6        1   7.465159
0      0.199327 -0.058239       7        1   7.015526
2     -0.266626 -0.038204       8        1   5.509059
6      0.249729  0.158927       9        1   5.156471
5      0.017953 -0.333223      10        1   3.785882, topic_info=       Term         Freq        Total Category  logprob  loglift
896     edu   614.000000   614.000000  Default  30.0000  30.0000
2724  space  1001.000000  1001.000000  Default  29.0000  29.0000
523     com   359.000000   359.000000  Default  28.0000  28.0000
399     car   524.000000   524.000000  Default  27.0000  27.0000
1216    god   425.000000   425.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
1714  makes    25.032935   152.088429  Topic10  -5.4689   1.4696
835    does    28.929816   568.808166  Topic10  -5.3242   0.2952
1230    gov    22.784822   108.915916  Topic10  -5.5630   1.7094
1516   john    23.085518   122.473569  Topic10  -5.5499   1.6052
3039    try    21.894465   189.121446  Topic10  -5.6028   1.1177

[665 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
3         1  0.025447           abs
3         3  0.941547           abs
9         1  0.176406  acceleration
9         3  0.141125  acceleration
9         8  0.035281  acceleration
...     ...       ...           ...
3226      6  0.985435         yeast
3229      1  0.077083          york
3229      4  0.067447          york
3229      5  0.144530          york
3229      8  0.713014          york

[2017 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 9, 5, 8, 10, 1, 3, 7, 6])

In [64]:
pyLDAvis.lda_model.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
1      -0.703557 -51.354378       1        1  18.910916
3     -33.286457 -46.206184       2        1  17.486832
8      32.184856 -54.067070       3        1  14.880483
4      52.021374 -23.343702       4        1  11.391817
7     -25.442108  19.079859       5        1   8.397854
9      39.935078  11.177523       6        1   7.465159
0       7.379580  16.078575       7        1   7.015526
2     -45.287952 -11.641207       8        1   5.509059
6     -11.660477 -15.802829       9        1   5.156471
5      18.359961 -19.386326      10        1   3.785882, topic_info=       Term         Freq        Total Category  logprob  loglift
896     edu   614.000000   614.000000  Default  30.0000  30.0000
2724  space  1001.000000  1001.000000  Default  29.0000  29.0000
523     com   359.000000   359.000000  Default  28.0000  28.0000
399     car   524.000000   524.000000  Default  27.0000  27.0000
1216    god   425.000000   425.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
1714  makes    25.032935   152.088429  Topic10  -5.4689   1.4696
835    does    28.929816   568.808166  Topic10  -5.3242   0.2952
1230    gov    22.784822   108.915916  Topic10  -5.5630   1.7094
1516   john    23.085518   122.473569  Topic10  -5.5499   1.6052
3039    try    21.894465   189.121446  Topic10  -5.6028   1.1177

[665 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
3         1  0.025447           abs
3         3  0.941547           abs
9         1  0.176406  acceleration
9         3  0.141125  acceleration
9         8  0.035281  acceleration
...     ...       ...           ...
3226      6  0.985435         yeast
3229      1  0.077083          york
3229      4  0.067447          york
3229      5  0.144530          york
3229      8  0.713014          york

[2017 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 9, 5, 8, 10, 1, 3, 7, 6])